In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ws = Workspace.from_config()
cluster_name = 'automlimagescompute'
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

In [ ]:
run_config = RunConfiguration()
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create()
run_config.environment.python.conda_dependencies.add_conda_package("numpy==1.18.5")
run_config.environment.python.conda_dependencies.add_conda_package("libffi=3.3")
run_config.environment.python.conda_dependencies.set_pip_requirements([
    "azureml-core==1.41.0",
    "azureml-mlflow==1.41.0",
    "azureml-dataset-runtime==1.41.0",
    "azureml-telemetry==1.41.0",
    "azureml-responsibleai==1.41.0",
    "azureml-automl-core==1.41.0",
    "azureml-automl-runtime==1.41.0",
    "azureml-train-automl-client==1.41.0",
    "azureml-defaults==1.41.0",
    "azureml-interpret==1.41.0",
    "azureml-train-automl-runtime==1.41.0",
    "azureml-automl-dnn-vision==1.41.0",
    "azureml-dataprep>=2.24.4"
])
run_config.environment.python.conda_dependencies.set_python_version('3.7')
run_config.environment.name = "AutoMLForImagesEnv"
run_config.environment.register(ws)

In [ ]:
model_name = PipelineParameter(name='model_name', default_value='Street_Segmentation_Model')
dataset_name = PipelineParameter(name='dataset_name', default_value='TRAIN_AML_Labeled_Street_Images')

In [ ]:
submit_job_step = PythonScriptStep(
    name='Submit AutoML Job',
    script_name='automl_job.py',
    arguments=[
        '--model_name', model_name,
        '--dataset_name', dataset_name,
        '--compute_name', 'automlimagescompute',
    ],
    compute_target=compute_target,
    source_directory='pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[submit_job_step])

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="AML Pipeline for training custom instance segmentation models using AutoML for Images.",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )


pipeline_endpoint_name = 'Instance Segmentation Model Training'
pipeline_endpoint_description = 'Sample pipeline for training and registering a custom instance segmentation model'

published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                     description=pipeline_endpoint_description,
                                     continue_on_step_failure=False)

published_pipeline_to_pipeline_endpoint(
    workspace=ws,
    published_pipeline=published_pipeline,
    pipeline_endpoint_name=pipeline_endpoint_name,
    pipeline_endpoint_description=pipeline_endpoint_description
)

In [ ]:
experiment = Experiment(ws, 'DEMO_AutoML_InstanceSegmentation')
run = experiment.submit(pipeline)